In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_io as tfio
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

from skimage import exposure, io, img_as_ubyte
from skimage.morphology import disk
from skimage.filters import rank
from skimage.restoration import denoise_bilateral

import tf_clahe
import os
import cv2
print(tf.__version__)

In [ ]:
def bcet(img):
    Lmin = tf.reduce_min(img) # MINIMUM OF INPUT IMAGE
#     Lmin = np.min(img) # MINIMUM OF INPUT IMAGE
#     print("Lmin", Lmin)
    Lmax = tf.reduce_max(img) # MAXIMUM OF INPUT IMAGE
#     Lmax = np.max(img) # MAXIMUM OF INPUT IMAGE
#     print("Lmax", Lmax)
    Lmean = tf.reduce_mean(img) #MEAN OF INPUT IMAGE
#     Lmean = np.mean(img) #MEAN OF INPUT IMAGE
#     print("Lmean", Lmean)
    LMssum = tf.reduce_mean(img * img) #MEAN SQUARE SUM OF INPUT IMAGE
#     LMssum = np.mean(pow(img, 2)) #MEAN SQUARE SUM OF INPUT IMAGE
#     print("LMssum", LMssum)

    Gmin = tf.constant(0, dtype="float32") #MINIMUM OF OUTPUT IMAGE
    Gmax = tf.constant(255, dtype="float32") #MAXIMUM OF OUTPUT IMAGE
    Gmean = tf.constant(110, dtype="float32") #MEAN OF OUTPUT IMAGE
    
    subber = tf.constant(2, dtype="float32")
    
    # find b
    
    bnum = ((Lmax**subber)*(Gmean-Gmin)) - (LMssum*(Gmax-Gmin)) + ((Lmin**subber) *(Gmax-Gmean))
    bden = subber * ((Lmax*(Gmean-Gmin)) - (Lmean*(Gmax-Gmin)) + (Lmin*(Gmax-Gmean)))
    
    b = bnum/bden
    
    # find a
    a1 = Gmax-Gmin
    a2 = Lmax-Lmin
    a3 = Lmax+Lmin-(subber*b)
            
    a = a1/(a2*a3)
    
    # find c
    c = Gmin - (a*(Lmin-b)**subber)
    
    # Process raster
    y = a*((img - b)**subber) + c #PARABOLIC FUNCTION
    
    final = tf.clip_by_value(y, clip_value_min=0, clip_value_max=255)
    return y

In [ ]:
def bcet_processing(img,channels=3):
#     img = tf.make_tensor_proto(img,dtype="int64")
#     img = tf.make_ndarray(img)
#     print(img.shape)
    layers = []
    for i in range(channels):
        layer = img[:,:,i]
        layer = bcet(layer)
        layers.append(layer)
        
# #     print(red.shape)
#     blue = img[:,:,1]
# #     print(blue.shape)
#     green = img[:,:,2]
#     red = img[:,:,0]
# #     print(red.shape)
#     blue = img[:,:,1]
# #     print(blue.shape)
#     green = img[:,:,2]
# #     print(green.shape)
    
#     red = bcet(red)
# #     print(red.shape)
#     blue = bcet(blue)
# #     print(blue.shape)
#     green = bcet(green)
# #     print(green.shape)
    
#     final_image = np.stack((red, blue, green), axis=-1)
#     final_image = tf.convert_to_tensor(final_image, dtype=tf.int64) 
#     print(final_image.shape)
    final_image = tf.stack(layers, axis=-1)
#     print(final_image.shape)
    return final_image

In [ ]:
def convert_file_bcet(inputFile, outputFile):

    img = tf.io.read_file(inputFile)
    img = tf.io.decode_bmp(img, channels=3)
    # print(tf.rank(img))
    img = tf.cast(img, tf.float32)

    
    img = bcet_processing(img)
    
    tf.keras.utils.save_img(outputFile, img)

def convert_contrast(inputFile, outputFile):

    img = tf.io.read_file(inputFile)
    img = tf.io.decode_png(img, channels=1)
    # print(tf.rank(img))
    img = tf.cast(img, tf.float32)

    
    img =  tf.image.adjust_contrast(img, 0.25)

    tf.keras.utils.save_img(outputFile, img)

def convert_file_clahe(inputFile, outputFile):
    bgr = cv2.imread(inputFile)

    lab = cv2.cvtColor(bgr, cv2.COLOR_BGR2LAB)

    lab_planes = cv2.split(lab)

    clahe = cv2.createCLAHE(clipLimit=5.0,tileGridSize=(15,15))

    eq_channels = []
    for ch in lab_planes:
        eq_channels.append(clahe.apply(ch))

    eq_image = cv2.merge(eq_channels)

    # lab = cv2.merge(lab_planes)

    bgr = cv2.cvtColor(eq_image, cv2.COLOR_LAB2BGR)
    cv2.imwrite(outputFile, bgr)

def convert_grayscale(inputFile, outputFile):
    bgr = cv2.imread(inputFile)

    lab = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)

    cv2.imwrite(outputFile, lab)
    
def convert_file_global_equal(inputFile, outputFile):
    img = cv2.imread(inputFile)
    # img = skimage.io.imread(fname=inputFile)
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

    lab_planes = cv2.split(img)
    eq_channels = []
    for ch in lab_planes:
        eq_channels.append(exposure.equalize_hist(ch))
    
    eq_image = cv2.merge(eq_channels)
    # bgr = cv2.cvtColor(eq_image, cv2.COLOR_LAB2BGR)
    
    io.imsave(fname=outputFile, arr=img_as_ubyte(eq_image))

def convert_file_local_equal(inputFile, outputFile):
    img = cv2.imread(inputFile)
    # img = skimage.io.imread(fname=inputFile)
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    
    lab_planes = cv2.split(img)
    eq_channels = []
    selem = disk(30)
    for ch in lab_planes:
        eq_channels.append(rank.equalize(ch, selem=selem))
    
    eq_image = cv2.merge(eq_channels)
    bgr = cv2.cvtColor(eq_image, cv2.COLOR_LAB2BGR)
    
    io.imsave(fname=outputFile, arr=img_as_ubyte(img))
    
def convert_denoise(inputFile, outputFile):
    img = cv2.imread(inputFile)
    
    img = denoise_bilateral(img_as_ubyte(img), sigma_color=0.1, sigma_spatial=15, multichannel=True)
    
    io.imsave(fname=outputFile, arr=img_as_ubyte(img))

def convert_custom_method(inputFile, outputFile):
    img = tf.io.read_file(inputFile)
    img = tf.io.decode_bmp(img, channels=3)
    # print(tf.rank(img))
    img = tf.cast(img, tf.float32)

    img = tfio.experimental.color.rgb_to_bgr(img)
    img = tf.image.adjust_contrast(img, 11.)
    img = tf.image.adjust_hue(img, 11.)
    img = tf.image.adjust_gamma(img)
    img = tfa.image.median_filter2d(img)
    
    tf.keras.utils.save_img(outputFile, img)

def convert_image_format(inputFile, ext, dest_ext):
    
    if inputFile.endswith(ext): 
        prefix = inputFile.split(ext)[0]
        im = Image.open(inputFile)
        im.save(prefix + dest_ext) 


kernel_size = (20, 20)
def histo_equalize(img):
    lab_planes = cv2.split(img)
    eq_channels = []
    for ch in lab_planes:
        eq_channels.append(cv2.equalizeHist(ch))

    img = cv2.merge(eq_channels)

    img = cv2.cvtColor(img, cv2.COLOR_LAB2BGR)
    return img

def opening_ops(img):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernel_size)
    img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    return img

def closing_ops(img):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernel_size)
    img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
    return img

def gabor_filter(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    g_kernel = cv2.getGaborKernel(kernel_size, 8.0, np.pi/4, 10.0, 0.5, 0, ktype=cv2.CV_32F)
    img = cv2.filter2D(img, cv2.CV_8UC3, g_kernel)
    return img

def convert_custom_method_v2(inputFile, outputFile):
    img = cv2.imread(inputFile)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

    img = opening_ops(img)
    img = closing_ops(img)
    img = histo_equalize(img)
    img = gabor_filter(img)
    
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    cv2.imwrite(outputFile, img)

def convert_sobelx(inputFile, outputFile):
    img = cv2.imread(inputFile)
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    img = cv2.Sobel(img, cv2.CV_64F,1,0,ksize=5)
    # img = cv2.cvtColor(img, cv2.COLOR_LAB2BGR)
    cv2.imwrite(outputFile, img)
    # return img
    

def convert_sobelxy(inputFile, outputFile):
    img = cv2.imread(inputFile)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.GaussianBlur(img,(3,3),0)
    img = cv2.Sobel(img,cv2.CV_64F,1,0,ksize=5)
    img = cv2.Sobel(img,cv2.CV_64F,0,1,ksize=5)
    cv2.imwrite(outputFile, img)
    
def convert_custom_v3(inputFile, outputFile):
    img = cv2.imread(inputFile)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.GaussianBlur(img,(3,3),0)
    img = cv2.Sobel(img,cv2.CV_64F,1,0,ksize=5)
    img = cv2.Sobel(img,cv2.CV_64F,0,1,ksize=5)
    # img = cv2.cvtColor(img, cv2.COLOR_LAB2BGR)
    cv2.imwrite(outputFile, img)
    # return img

In [ ]:
## convert colour of images
for mode in ["train", "test"]:
    for class_name in ["normal", "defect"]:
        Input_dir = f'mura_data/RGB/mura_clean/{mode}_data/{class_name}/'
        Out_dir = f'mura_data/RGB/sobelxy_clean/{mode}_data/{class_name}/'
        a = os.listdir(Input_dir)
        for i in a:
            if i != ".DS_Store":

                inputFile = Input_dir+i
                OutputFile = Out_dir+i

                # print(inputFile)
                # convert_file_bcet(inputFile, OutputFile)
                # convert_file_clahe(inputFile, OutputFile)
                # convert_sobelx(inputFile, OutputFile)
                convert_sobelxy(inputFile, OutputFile)
                # convert_custom_method_v2(inputFile, OutputFile)
                # convert_file_clahe_v2(inputFile, OutputFile)
                # convert_image_format(inputFile, ".bmp", ".png")
                # convert_custom_method(inputFile, OutputFile)
                # convert_file_global_equal(inputFile, OutputFile)
                # convert_file_local_equal(inputFile, OutputFile)
                # convert_grayscale(inputFile, OutputFile)
                # convert_contrast(inputFile, OutputFile)
                # convert_denoise(inputFile, OutputFile)
                # convert_custom_v3(inputFile, OutputFile)

        print("done.", class_name, mode)
    #     break